## 간단한 Visdom 사용법

In [2]:
import torch
import torch.nn as nn

import torchvision
import torchvision.datasets as dsets

import visdom
vis = visdom.Visdom()

Setting up a new session...


In [3]:
# Visdom에 text 출력
vis.text('Hello, world!', env='main')

'window_387d48063c23be'

In [5]:
# Visdom에 img 출력
a = torch.randn(3, 200, 200)
vis.image(a)

'window_387d485c57530e'

In [7]:
MNIST = dsets.MNIST(root="./MNIST_data",train = True,transform=torchvision.transforms.ToTensor(), download=True)
cifar10 = dsets.CIFAR10(root="./cifar10",train = True, transform=torchvision.transforms.ToTensor(),download=True)

Extracting ./cifar10\cifar-10-python.tar.gz to ./cifar10


## Cheaking Dataset with Visdom

In [8]:
# CIFAR10
data = cifar10.__getitem__(0)
print(data[0].shape)
vis.images(data[0], env="main")

torch.Size([3, 32, 32])


'window_387d48d04e502e'

In [9]:
data = MNIST.__getitem__(0)
print(data[0].shape)
vis.images(data[0], env="main")

torch.Size([1, 28, 28])


'window_387d48d926d006'

In [19]:
# DataLoader를 통한 dataload
data_loader = torch.utils.data.DataLoader(dataset = MNIST,
                                         batch_size = 32,
                                         shuffle = False)

In [20]:
for num, value in enumerate(data_loader):
    value = value[0]
    print(value.shape)
    vis.images(value)
    break

torch.Size([32, 1, 28, 28])


In [40]:
# visdom의 모든 창을 닫는다.
vis.close(env="main")

''

### Line Plot

In [41]:
Y_data = torch.randn(5) #만약 X값을 넣지 않으면 0~1 사이의 point의 갯수를 의미하게 된다.
plt = vis.line (Y=Y_data)

In [42]:
# X의 index 추가
X_data = torch.Tensor([1, 2, 3, 4, 5])
plt = vis.line(Y=Y_data, X=X_data)

In [48]:
# Line update
Y_append = torch.randn(1)
X_append = torch.Tensor([6])

vis.line(Y=Y_append, X=X_append, win = plt, update='append')

'window_387d4a70ff6a6e'

In [49]:
# Multiple Line on single windows
num = torch.Tensor(list(range(0, 10)))
num = num.view(-1, 1)
num = torch.cat((num,num), dim=1)

plt = vis.line(Y=torch.randn(10, 2), X = num)

### Line info

In [50]:
# graph 이름 설정
plt = vis.line(Y=Y_data, X=X_data, opts = dict(
    title='Test', showlegend=True)
              )

In [51]:
plt = vis.line(Y=Y_data, X=X_data, opts = dict(
    title='Test', legend = ['1번'],showlegend=True)
              )

In [52]:
# legend 이름 설정
plt = vis.line(Y=torch.randn(10,2), X = num, opts=dict(
    title='Test', legend=['1번','2번'],showlegend=True
))

### make function for update line

In [53]:
def loss_tracker(loss_plot, loss_value, num):
    '''num, loss_value -> !!Tensor Type!!'''
    vis.line(X=num,
            Y=loss_value,
            win = loss_plot,
             update = 'append'
            )

In [54]:
plt = vis.line(Y=torch.Tensor(1).zero_())

for i in range(500):
    loss = torch.randn(1) + i
    loss_tracker(plt, loss, torch.Tensor([i]))

In [55]:
vis.close(env="main")

''

# MNIST-CNN with Visdom

In [56]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms

import torch.nn.init


import visdom

vis = visdom.Visdom()
vis.close(env="main")

Setting up a new session...


''

In [62]:
def loss_tracker(loss_plot, loss_value, num):
    '''num, loss_value, are Tensor'''
    vis.line(X=num,
             Y=loss_value,
             win = loss_plot,
             update='append'
             )

In [93]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device =='cuda':
    torch.cuda.manual_seed_all(777)

In [101]:
#parameters
learning_rate = 0.001
training_epochs = 10
batch_size = 800

In [102]:

#MNIST dataset

mnist_train = dsets.MNIST(root='MNIST_data/',
                         train = True,
                         transform=transforms.ToTensor(),
                         download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                        train=False,
                        transform = transforms.ToTensor(),
                        download=True)

In [103]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size = batch_size,
                                          shuffle =True,
                                          drop_last=True)

In [104]:
# CNN Model (2 conv layers)
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        # L1 ImgIn shape=(?, 28, 28, 1)
        #    Conv     -> (?, 28, 28, 32)
        #    Pool     -> (?, 14, 14, 32)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        # L2 ImgIn shape=(?, 14, 14, 32)
        #    Conv      ->(?, 14, 14, 64)
        #    Pool      ->(?, 7, 7, 64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        # Final FC 7x7x64 inputs -> 10 outputs
        self.fc = torch.nn.Linear(7 * 7 * 32, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)   # Flatten them for FC
        out = self.fc(out)
        return out

In [105]:
model = CNN().to(device)

value = (torch.Tensor(1,1,28,28)).to(device)
print( (model(value)).shape )


criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

torch.Size([1, 10])


In [106]:
# Make plot
loss_plt = vis.line(Y=torch.Tensor(1).zero_(),opts=dict(title='loss_tracker', legend=['loss'], showlegend=True))

In [107]:
#training
total_batch = len(data_loader)
print('Learning Started!')

for epoch in range(training_epochs):
    avg_cost = 0
    
    for X, Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
    
    print('[Epoch:{}] cost = {}'.format(epoch+1, avg_cost))
    # Loss_tracker 실행. epoch가 끝날 때 마다 avg_cost를 입력
    loss_tracker(loss_plt, torch.Tensor([avg_cost]), torch.Tensor([epoch]))
print('Learning Finished!')

Learning Started!
[Epoch:1] cost = 1.0378667116165161
[Epoch:2] cost = 0.2466728836297989
[Epoch:3] cost = 0.1576475352048874
[Epoch:4] cost = 0.1129569560289383
[Epoch:5] cost = 0.09020482003688812
[Epoch:6] cost = 0.07633184641599655
[Epoch:7] cost = 0.06634559482336044
[Epoch:8] cost = 0.060921452939510345
[Epoch:9] cost = 0.05550011992454529
[Epoch:10] cost = 0.05166101083159447
Learning Finished!


![image.png](./img/10_02_visdom.png)

In [109]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean() 
    print('Accuracy:', accuracy.item())

Accuracy: 0.9467999935150146
